In [7]:
import pandas as pd
import nltk
import operator
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition

In [8]:
df = pd.read_json('/Users/ivanshadrin/Documents/ivan-shadrin/ivan-shadrin/Documents/Ventuary/gcloud/back.json', lines=True)

### Датасет представляет собой данные о блокчейн-стартапах выгруженные через API с сайта https://icobench.com/

In [20]:
df.head()

,accepting,available_for_sale,avatar_url,average_industry_hard_cap,bench_general_score,connected_peoples,connected_projects,end_date,founders_residency,hard_cap,...,tags,team_size,ticker,title,token_type,total_tokens,ventuary_investment_score,ventuary_score,ventuary_team_score,whitepaper_url
website_url,,,,,,,,,,,,,,,,,,,,,
https://adhive.tv/?utm_source=icobench&utm_medium=listing&utm_campaign=ico,"ETH, BTC, Fiat",0,https://icobench.com/images/icos/icons/adhive.jpg,0,0,NaN,NaN,2018-03-14,,0,...,"[marketing, cloud storage, AI & ML]",10,ADH,AdHive,ERC20,0,0,0,0,https://adhive.tv/docs/AdHive_Whitepaper.pdf
https://tokensale.shping.com/?utm_source=icobench,"BTC, ETH, ETC, LTC, DASH, ZEC, Fiat",0,https://icobench.com/images/icos/icons/shping.jpg,0,0,NaN,NaN,2018-03-23,,0,...,"[innovation, VR & entertainment, software]",16,SHPING,Shping,ERC20,0,0,0,0,https://tokensale.shping.com/assets/shping_coi...
https://socialmedia.market/?utm_source=icobench,"ETH, BTC, LTC, XRP, BCH, DASH, DOGE, ETC, NEO,...","40,000,000",https://icobench.com/images/icos/icons/socialm...,0,0,NaN,NaN,2018-03-16,,0,...,"[social media, marketing, trading]",26,SMT,SocialMedia.Market,0,"40,000,000",0,0,0,https://socialmedia.market/assets/pdf/SMM_Whit...
https://tokenstars.com/team/?utm_source=icobench,"BTC, ETH, ACE","45,000,000",https://icobench.com/images/icos/icons/tokenst...,0,0,NaN,NaN,2018-03-28,,0,...,"[exchanges, marketing, trading]",11,TEAM,TokenStars TEAM,ERC20,"45,000,000",0,0,0,https://tokenstars.com/upload/files/team_white...
https://vinchain.io/en?utm_source=icobench,"ETH, BTC, LTC, DASH","600,000,000",https://icobench.com/images/icos/icons/vinchai...,0,0,NaN,NaN,2018-04-15,,0,...,"[data, innovation, health & medicine]",20,VIN,VinChain,ERC20,"600,000,000",0,0,0,https://vinchain.io/files/VinChainWhitePaper.pdf


### Задача - разделить стартапы на категории и получить значения категорий для каждого проекта

In [9]:
tokentext = df["short_description"].map(str) + df["long_description"]
freqstop=['blockchain',
 'platform',
 'token',
 'tokens',
 'market',
 'technology',
 'users',
 'world',
 'network',
 'based',
 'new',
 'services',
 'smart',
 'use',
 'crypto',
 'time',
 'digital',
 'ico',
 'value',
 'decentralized',
 'exchange',
 'project',
 'people',
 'business',
 'currency',
 'user',
 'create',
 'access',
 'ecosystem',
 'service',
 'investors',
 'community',
 'provide',
 'contracts',
 'industry',
 'transactions',
 'global',
 'coin',
 'way',
 'money',
 'able',
 'development',
 'transaction',
 'cryptocurrencies',
 'company',
 'assets',
 'contract',
 'information',
 'online',
 'team',
 'allows',
 'companies',
 'high',
 'future',
 'like',
 'funds',
 '000',
 'price',
 'projects',
 'process',
 'solution',
 'secure',
 'products',
 'peer',
 'sale',
 'open',
 'experience',
 'need',
 'asset',
 'help',
 'available',
 'developers',
 'app',
 'creating',
 'fees',
 'customers',
 'work',
 'product',
 'currencies',
 'application',
 'unique',
 'coins',
 'allow',
 'management']
freqstop += ["around","want","become","get","every","well","within","without","created","provides","new","uses","improve","making","made","powered","launch","first","one","using","make","also","system","used"]
stopset = set(nltk.corpus.stopwords.words('english'))
stopset.update(freqstop)
stopset.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

### После добавления стоп-слов, воспользуемся TF-IDF, о котором мы говорили на посоеднем занятии

In [10]:
tfidf = TfidfVectorizer(stop_words=stopset, min_df = 15)
A = tfidf.fit_transform(tokentext)

In [11]:
terms = tfidf.get_feature_names()
def rank_terms( A, terms ):
    sums = A.sum(axis=0)
    weights = {}
    for col, term in enumerate(terms):
        weights[term] = sums[0,col]
    return sorted(weights.items(), key=operator.itemgetter(1), reverse=True)
ranking = rank_terms( A, terms )
k=32

model = decomposition.NMF( init="nndsvd", n_components=k ) 

W = model.fit_transform( A )
H = model.components_


In [12]:
colnames1 = []

for i in range(32):
    colnames1.append("topic_"+str(i))

colnames1.append("name")
featuredf1 = pd.DataFrame(columns=colnames1)

### Применим Non-Negative Matrix Factorization к описаниям каждого проекта и получим таблицу с фичами для кажлой категории для кадого проекта

In [13]:
for i, row in df.iterrows():

    featuredf1.loc[i,'name'] = row.website_url
    #print(row.token_name)
    text = tokentext[i]
    x = model.transform(tfidf.transform([text]))[0]
    x = str(x).strip("[]").split()
    x = [float(num) for num in x]
    
    for topic_num, score in enumerate(x):
        featuredf1.loc[i, "topic_"+str(topic_num)] = score

featuredf1 = featuredf1.set_index('name')
featuredf1.loc['https://adhive.tv/?utm_source=icobench&utm_medium=listing&utm_campaign=ico']

topic_0               0
topic_1               0
topic_2               0
topic_3      0.00267247
topic_4     0.000895433
topic_5     0.000106256
topic_6      0.00302313
topic_7               0
topic_8        0.008625
topic_9               0
topic_10     0.00240829
topic_11              0
topic_12      0.0565936
topic_13    0.000627454
topic_14              0
topic_15       0.219977
topic_16              0
topic_17              0
topic_18              0
topic_19     0.00158899
topic_20              0
topic_21              0
topic_22              0
topic_23              0
topic_24      0.0864648
topic_25     0.00315436
topic_26              0
topic_27    0.000772018
topic_28     0.00667695
topic_29              0
topic_30       0.003104
topic_31              0
Name: https://adhive.tv/?utm_source=icobench&utm_medium=listing&utm_campaign=ico, dtype: object

In [14]:
featuredf1.columns = ['credit', 'data', 'real estate', 'media & content', 'gambling', 'cryptocurrency', 'exchanges', 'mining', 'investment', 'social media', 'ethereum', 'energy', 'AI & ML', 'mobile', 'payments', 'marketing', 'financial', 'health & medicine', 'gaming', 'hedging', 'betting', 'marketplace', 'travel', 'platform', 'cloud storage', 'VR & entertainment', 'software', 'loyalty program', 'innovation', 'identity verification', 'video', 'trading']

In [15]:
featuredf1

,credit,data,real estate,media & content,gambling,cryptocurrency,exchanges,mining,investment,social media,...,travel,platform,cloud storage,VR & entertainment,software,loyalty program,innovation,identity verification,video,trading
name,,,,,,,,,,,,,,,,,,,,,
https://adhive.tv/?utm_source=icobench&utm_medium=listing&utm_campaign=ico,0,0,0,0.00267247,0.000895433,0.000106256,0.00302313,0,0.008625,0,...,0,0,0.0864648,0.00315436,0,0.000772018,0.00667695,0,0.003104,0
https://tokensale.shping.com/?utm_source=icobench,0.00178944,0.00170443,0.00056576,0,0,0.00636688,0,0.00449585,0.00367427,0,...,0,0.0042605,0,0.0461016,0.00833204,0,0.0678758,0,0,0.00408219
https://socialmedia.market/?utm_source=icobench,0,0,0,0.00379363,0,0,0.00132456,0,0.0010182,0.253296,...,0,0,0,0,0,0,0,0,0.0131401,0.0142942
https://tokenstars.com/team/?utm_source=icobench,0.0034608,0.00766029,0.00151027,0.00370849,0.00369553,0,0.0709198,0,0,0.00249252,...,0.00919089,0.00296582,0.00521238,0.00622299,0,0.0025159,0,0.00993933,0.00710203,0.0217214
https://vinchain.io/en?utm_source=icobench,6.25218e-05,0.163124,0,0.00276525,0.000597371,0,0.00226889,0.00255007,0,0,...,0.00586351,0.00278468,0.00780169,0.000188389,0,0.00427274,0.121741,0.0103311,0,0
https://gpcc.io/?utm_source=icobench,0.00279035,0,0.0111268,0.0129339,0,0.00349591,0,0.00287146,0,0.0367483,...,0.00528475,0,0.0478769,0,0.00049258,0,0.00503681,0,0,0.0288853
https://www.helbizcoin.io/?utm_source=icobench,0,0,0,0,0,8.6191e-05,0.000182796,0,0,0.000670292,...,0,0.0151514,0,0.00140078,0.00331903,0,0,0,0,0.0158459
https://iqeon.io/?utm_source=icobench,0,0,0,0,0.0275716,0,0.00073812,0,0.00973891,0.00228203,...,0,0,0,0,0,0,0.0236746,0.0729613,0.0136525,0
https://www.lympo.io/?utm_source=icobench,0,0.129494,0.00025495,0,0,0,0,0,0,0,...,0,0,0,0,0.00982184,0,0,0,0,0


### Категории получены

In [16]:
newframe1 = pd.DataFrame(columns=['closest_topics'])
for i, row in featuredf1.iterrows():
    newframe1.at[i,'closest_topics'] = row.sort_values(ascending=False)[:3].index.tolist()
newframe1

,closest_topics
https://adhive.tv/?utm_source=icobench&utm_medium=listing&utm_campaign=ico,"[marketing, cloud storage, AI & ML]"
https://tokensale.shping.com/?utm_source=icobench,"[innovation, VR & entertainment, software]"
https://socialmedia.market/?utm_source=icobench,"[social media, marketing, trading]"
https://tokenstars.com/team/?utm_source=icobench,"[exchanges, marketing, trading]"
https://vinchain.io/en?utm_source=icobench,"[data, innovation, health & medicine]"
https://gpcc.io/?utm_source=icobench,"[cloud storage, social media, trading]"
https://www.helbizcoin.io/?utm_source=icobench,"[marketplace, trading, platform]"
https://iqeon.io/?utm_source=icobench,"[identity verification, betting, gambling]"
https://www.lympo.io/?utm_source=icobench,"[data, health & medicine, betting]"
https://www.minerone.io/?utm_source=icobench,"[mining, cloud storage, travel]"


In [17]:
df = df.set_index('website_url')
df['tags'] = newframe1['closest_topics']